**Universidad Internacional de La Rioja (UNIR) - Máster Universitario en Inteligencia Artificial - Procesamiento del Lenguaje Natural** 

***
Datos del alumno (Nombre y Apellidos): Daniel Sabbagh Pastor

Fecha: 28/04/2024
***

<span style="font-size: 20pt; font-weight: bold; color: #0098cd;">Trabajo: Named-Entity Recognition</span>

**Objetivos** 

Con esta actividad se tratará de que el alumno se familiarice con el manejo de la librería spacy, así como con los conceptos básicos de manejo de las técnicas NER

**Descripción**

En esta actividad debes procesar de forma automática un texto en lenguaje natural para detectar características básicas en el mismo, y para identificar y etiquetar las ocurrencias de conceptos como localización, moneda, empresas, etc.

En la primera parte del ejercicio se proporciona un código fuente a través del cual se lee un archivo de texto y se realiza un preprocesado del mismo. En esta parte el alumno tan sólo debe ejecutar y entender el código proporcionado.

En la segunda parte del ejercicio se plantean una serie de preguntas que deben ser respondidas por el alumno. Cada pregunta deberá responderse con un fragmento de código fuente que esté acompañado de la explicación correspondiente. Para elaborar el código solicitado, el alumno deberá visitar la documentación de la librería spacy, cuyos enlaces se proporcionarán donde corresponda.

# Parte 1: carga y preprocesamiento del texto a analizar

Observa las diferentes librerías que se están importando.

In [4]:
import pathlib
import spacy
import pandas as pd
from spacy import displacy
import csv
import es_core_news_md

El siguiente código simplemente carga y preprocesa el texto. Para ello, lo primero que hace es cargar un modelo de lenguaje previamente entrenado. En este caso, se utiliza <i>es_core_news_md</i>: 

https://spacy.io/models/es#es_core_news_md


In [5]:
nlp = es_core_news_md.load()

El objeto <i>nlp</i> permite utilizar el modelo de lenguaje cargado, de forma que se puede procesar un texto y obtenerlo en su versión preprocesada. Así, nos permite realizar las diferentes tareas. En este caso, vamos a utilizar el pipeline para hacer un preprocesamiento básico, que consiste en tokenizar el texto.

In [6]:
filename = "./comentariosOdio.csv"
#Lectura del csv, selecciono la opción de que no me corte las columnas que me muestra, para visualizar el dataframe entero
pd.set_option('display.max_colwidth', None)
#Ha habido bastantes problemas para leer el archivo, lo que hice fue abrirlo como archivo txt con notepad y volverlo a guardar como archivo UTF-8
#Para posteriormente leerlo como UTF-8 e ignorando los casos en los que haya errores
data = pd.read_csv(filename, delimiter=';', encoding="utf-8", encoding_errors='ignore')  
data['CONTENIDO A ANALIZAR'].head(10)

C:\Users\Dani\AppData\Local\Temp\ipykernel_18672\238657758.py:6: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename, delimiter=';', encoding="utf-8", encoding_errors='ignore')


0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      el barça nunca acaeza ante un segundo b ni ante un tercera , ya estan los arbitro

El código anterior carga el archivo CSV (opcionalmente con un límite de líneas a leer) y genera la variable <i>data</i>, que contiene un Dataframe (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) con los datos leídos del CSV.

Te vendrá bien conocer la siguiente documentación:
<ul>
    <li>https://spacy.io/api/doc</li>
    <li>https://spacy.io/api/token</li>
    <li>https://spacy.io/api/morphology#morphanalysis</li>
</ul>

### Playground

Utiliza este espacio para hacer pruebas y ensayos con las variables generadas con el código previo. A modo de ejemplo, se ofrece código que realiza las siguientes tareas: 


- leer un número dado de líneas del Dataframe y generar dos listas con los valores (se pueden leer directamente del DataFrame, se muestra el ejemplo como una opción más)
- procesar el texto de cada comentario


Para procesarlo, hay utilizar el objeto <i>nlp</i> y así obtener objetos de la clase <i>Doc</i> (https://spacy.io/api/doc)

Visita la documentación de dicha clase y experimenta probando las diferentes funciones y atributos 

In [7]:
############
##LIMPIEZA##
############
#Debido a que la extensión del dataframe era de 16 columnas, quise limpiarlo para que solo me quedara información útil.
#De la columna 9 a la 15, eran llamadas Unnamed y únicamente con valores NaN
#Con el bucle busco demostrar cuantos valores nulos hay en cada columna para comparar la diferencia frente al total

for a in range(9,16):
    nulos = data[f'Unnamed: {a}'].isnull().sum()
    print(f"Nombre Columna: Unnamed {a}")
    print("Nulos", nulos)
    print("Totales",len(data))
    print("No nulos: ", (len(data)-nulos))
#Después de comprobar que son valores NaN y columnas unnamed con muchos valores despreciables, procedo a eliminarlas
for b in range(9,16):
    data = data.drop(f'Unnamed: {b}', axis=1)

#Obteniendo los valores únicos de la columna Intensidad, visualizo que hay 3 registros que solo se usan una vez cada uno, por lo que parecía haber 3 registros del #dataset que estarían en la posición equivocada, por lo que, sabiendo cuales son, procedo a borrarlos (eran esos dos textos que parecen calles o ubicaciones y un #float) 
valoresNoDeseados = [" la vila joiosa, 52", " tanya kozyreva en ucrania", 14] 
filasTotales = len(data)
data = data[~data['INTENSIDAD'].isin(valoresNoDeseados)]
filasBorradas = filasTotales - len(data)
print(f"Se han borrado {filasBorradas} filas.")
conteoValores = data['INTENSIDAD'].value_counts()

pctValores = (conteoValores / len(data)) * 100
print(pctValores)

#Uso de la función dropNa de pandas para eliminar los valores NaN de las columnas CONTENIDO A ANALIZAR y INTENSIDAD
data = data.dropna(subset=['CONTENIDO A ANALIZAR', 'INTENSIDAD'])
#Se requiere crear una columna que pueda determinar si Hay o no Hay Odio, por lo que se procede a crear una función que asigne True cuando el valor de intensidad #es >0 y por el contrario False cuando sea 0
#Por lo que primero se convierte a valores numéricos para aplicarle la condición correctamente.

data['INTENSIDAD'] = pd.to_numeric(data['INTENSIDAD'], errors='coerce')
def calcular_odio(valor):
    if pd.isnull(valor):
        return False  #Si encuentra valores nulos retornará False, se entiende que no hay odio si encuentra valores creados por el parámetro coerce (NaN o NaT)
    return valor > 0

#Aplico la función calcular_odio a la columna 'INTENSIDAD' y crea la nueva columna 'ODIO'
data['ODIO'] = data['INTENSIDAD'].apply(calcular_odio)

data['ODIO'].head(10)

#####################
##REDUCCIÓN DATASET##
#####################
#Debido a que las ejecuciones de cada ejercicio tardaban entre 90 y 110 minutos he procedido quedarme con un 20% de los datos, eso sí, buscando siempre que me #quede la misma proporción de datos con respecto a la nueva columna ODIO.
#Los resultados de ambas ejecuciones, tanto con el 100% de los datos como con el 20% son resultados muy parecidos. Al final de esta celda de ejecución se mostrarán los resultados con las ejecuciones completas.

#Se busca crear una máscara para cada parte del contenido
dataOdio = data[data['ODIO'] == True]
dataNoOdio = data[data['ODIO'] == False]

Odio20 = dataOdio.iloc[:len(dataOdio) // 5]
NoOdio20 = dataNoOdio.iloc[:len(dataNoOdio) // 5]

#Como se han dividido odio y no odio por separado procedo a concatenarlos.
data_20 = pd.concat([Odio20, NoOdio20])
#El nuevo archivo data_20 contiene el 20% de los valores que contenía el antiguo

print("Número de registros en la mitad de los datos con 'ODIO' True:", len(data_20[data_20['ODIO']==True]))
print("Número de registros en la mitad de los datos con 'ODIO' False:", len(data_20[data_20['ODIO']==False]))




Nombre Columna: Unnamed 9
Nulos 574912
Totales 574915
No nulos:  3
Nombre Columna: Unnamed 10
Nulos 574912
Totales 574915
No nulos:  3
Nombre Columna: Unnamed 11
Nulos 574913
Totales 574915
No nulos:  2
Nombre Columna: Unnamed 12
Nulos 574913
Totales 574915
No nulos:  2
Nombre Columna: Unnamed 13
Nulos 574913
Totales 574915
No nulos:  2
Nombre Columna: Unnamed 14
Nulos 574913
Totales 574915
No nulos:  2
Nombre Columna: Unnamed 15
Nulos 574913
Totales 574915
No nulos:  2
Se han borrado 3 filas.
0.0    86.591339
0.0    11.222413
4.0     1.675909
1.0     0.229774
4.0     0.128715
5.0     0.035310
3.0     0.029570
2.0     0.025569
1.0     0.009741
6.0     0.001739
5.0     0.001392
6.0     0.000522
3.0     0.000348
2.0     0.000174
Name: INTENSIDAD, dtype: float64
Número de registros en la mitad de los datos con 'ODIO' True: 2459
Número de registros en la mitad de los datos con 'ODIO' False: 112468


In [8]:
###############################################
##TOKENIZACIÓN, LEMMA, STOPWORDS Y PUNTUACIONES##
###############################################

#NECESITO CREAR ESTA CELDA PARA EJECUTAR POR SEPARADO LA LIMPIEZA.
#Se genera la limpieza del contenido, buscando separar en tokens y quitar signos de puntuación y palabras vacías de significado.
def limpiezaContenido(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

data_20['CONTENIDO A ANALIZAR TOKENIZED'] = data_20['CONTENIDO A ANALIZAR'].apply(limpiezaContenido)
data_20['CONTENIDO A ANALIZAR TOKENIZED'].head(50)



0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      barça acaezar b arbitro impedir él messi autentico vergüenza
2                                                                                                                                                                                                                                              cristín cifuent presidenta madrid añorar madrileño vista ayuso llegar presidenta pp m

In [ ]:
#################################################
##RESULTADOS CON EL 100% DE VALORES DEL DATASET##
#################################################
'''
Pregunta 1:
  Output:
    Número de registros en el corpus: 574639
    Número de registros en el corpus: 574639
Pregunta 2:
  Output:
    El número total de palabras es: 62405613
Pregunta 3:
  Output:
    Hay 108.6 palabras de media en cada comentario
Pregunta 4:
  Output:
    Promedio de palabras en comentarios de odio: 15.96
    Promedio de palabras en comentarios de no odio: 110.63
Pregunta 5:
  Output:
    Promedio de oraciones en comentarios de odio: 1.56
    Promedio de oraciones en comentarios de no odio: 3.93
Pregunta 6:
  Output:
    Porcentaje de comentarios con entidades NER en comentarios de odio: 34.3
    Porcentaje de comentarios con entidades NER en comentarios de no odio: 58.09
Pregunta 7:
  Output:
    Porcentaje de comentarios con entidades NER de tipo PERSON en comentarios de odio: 18.88
    Porcentaje de comentarios con entidades NER de tipo PERSON en comentarios de no odio: 28.85
Pregunta 8:
  Output:
    Porcentaje de palabras en cada combinación posible en comentarios de odio:
      ('Masc', 'Sing'): 29.62
      ('Masc', 'Plur'): 15.23
      ('Fem', 'Sing'): 34.15
      ('N', 'S'): 6.37
      ('Fem', 'Plur'): 8.55
      ('N', 'Plur'): 3.68
      ('N', 'Sing'): 1.84
      ('Masc', 'S'): 0.36
      ('Fem', 'S'): 0.21

    Porcentaje de palabras en cada combinación posible en comentarios de no odio:
      ('Masc', 'Sing'): 29.95
      ('Fem', 'Sing'): 31.75
      ('N', 'S'): 8.84
      ('Fem', 'Plur'): 10.9
      ('Masc', 'Plur'): 14.24
      ('N', 'Sing'): 1.9
      ('Fem', 'S'): 0.37
      ('N', 'Plur'): 1.67
      ('Masc', 'S'): 0.39

Pregunta 9:
  Output:
    Entidades por tipo en comentarios de odio:
      'PER': 2798
      'LOC': 1399 
      'MISC': 1047
      'ORG': 624

    Entidades por tipo en comentarios de no odio:
      'LOC': 730972
      'PER': 690686
      'ORG': 212861
      'MISC': 208855
Pregunta 10: El Output se recorta a 5 lemas de cada grupo:
  Output:
    Comentarios de odio:
      mierda - 866
      puta - 782
      asco - 575
      gobierno - 542
      panfleto - 487
    Comentarios de no odio:
      año - 171496
      gobierno - 87342
      persona - 83658
      país - 77163
      caso - 72298
'''

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 1.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuántos registros contiene el corpus?</span>

In [9]:
# Incluye aquí el código generado para poder responder a tu pregunta
#Forma 1:
totalRows = len(data_20)
print("Número de registros en el corpus:", totalRows)
#Forma 2:
totalRows2 = data_20.shape[0]
print("Número de registros en el corpus:", totalRows2)


Número de registros en el corpus: 114927
Número de registros en el corpus: 114927


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
<p>En este fragmento de código, se está determinando el número de entradas en el corpus. Se han proporcionado dos métodos distintos para obtener esta información:</p>
<ul>
  <li>Método 1: Se emplea la función len() para calcular la longitud de data_20, que corresponde al número de entradas en el corpus.</li>
  <li>Método 2: Se utiliza la propiedad shape[0] de data_20 para acceder al número de filas en el corpus, lo cual es igual al número de entradas.</li>
</ul>
<p>El resultado final en todos los casos es idéntico, demostrando que el corpus contiene un total de 114,927 entradas.</p>
 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 2.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuántas palabras totales hay en los comentarios del corpus?</span>

In [12]:
# Incluye aquí el código generado para poder responder a tu pregunta

comentariosTexto = data_20['CONTENIDO A ANALIZAR TOKENIZED'].apply(lambda x: str(x))

totalPalabras = comentariosTexto.apply(lambda x: len(x.split())).sum()

print(f"El número total de palabras es: {totalPalabras}")


El número total de palabras es: 6518080


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>

<ol>En este fragmento de código, se está calculando la cantidad total de palabras en los comentarios del conjunto de datos. Aquí se presenta una explicación detallada:</p>
<ol>
  <li>Se accede a la columna 'CONTENIDO A ANALIZAR TOKENIZED' del DataFrame data_20, la cual contiene los comentarios tokenizados.</li>
  <li>Se aplica una función lambda a cada elemento de esta columna para asegurarse de que todos los valores sean tratados como cadenas de texto mediante str(x).</li>
  <li>Posteriormente, se emplea otra función lambda en cada comentario tokenizado para dividirlo en palabras utilizando el método split().</li>
  <li>Luego, se utiliza el método sum() para sumar el número total de palabras en todos los comentarios.</li>
</ol>

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 3.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuál el número promedio de palabras en cada comentario?</span>

In [13]:
# Incluye aquí el código generado para poder responder a tu pregunta
#TotalPalabras es una variable del ejercicio 2
commentsAVG = round(totalPalabras / totalRows,3)
print(f"Hay {commentsAVG} palabras de media en cada comentario" )

Hay 56.715 palabras de media en cada comentario


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>

<p>Se divide la cantidad total de palabras (totalPalabras) entre la cantidad total de registros en el conjunto de datos (totalRegistros). Esto se hace para calcular el promedio de palabras por comentario.</p>
<p>El resultado se aproxima a tres decimales utilizando la función round() para obtener un valor más fácil de leer.</p>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 4.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál el número promedio de palabras en los comentarios de cada grupo?</span>

In [14]:

#####
## A partir del ejercicio siguiente se usarán bastante las variables "comentariosOdio" y "comentariosNoOdio"
#####
comentariosOdio = data_20[data_20['ODIO'] == True]
comentariosNoOdio = data_20[data_20['ODIO'] == False]

#Función que devuelve el número de palabras de un comentario
def palabrasPorGrupo(comentario):
    palabras = comentario.split()
    return len(palabras)

mediaPalabrasOdio = comentariosOdio['CONTENIDO A ANALIZAR TOKENIZED'].apply(palabrasPorGrupo).mean()
mediaPalabrasOdio = comentariosNoOdio['CONTENIDO A ANALIZAR TOKENIZED'].apply(palabrasPorGrupo).mean()

print("Promedio de palabras en comentarios de odio:", mediaPalabrasOdio)
print("Promedio de palabras en comentarios de no odio:", mediaPalabrasOdio)


Promedio de palabras en comentarios de odio: 57.781288899953765
Promedio de palabras en comentarios de no odio: 57.781288899953765


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>

<p>Se segregan los comentarios en dos grupos distintos del corpus utilizando la columna 'ODIO'. Se crea un DataFrame llamado comentariosOdio que contiene los comentarios etiquetados como negativos (valor True en la columna 'ODIO') y otro DataFrame llamado comentariosNoOdio que incluye los comentarios no etiquetados como negativos (valor False en la columna 'ODIO').</p>
<p>Se establece una función denominada palabrasPorGrupo(comentario) que recibe un comentario como entrada y devuelve el número de palabras presentes en él. Esta función divide el comentario en palabras utilizando el método split() y luego determina la longitud de la lista resultante.</p>
<p>Posteriormente, se aplica la función ContarPalabrasPorGrupo a la columna 'CONTENIDO A ANALIZAR TOKENIZED' tanto del DataFrame comentariosOdio como del comentariosNoOdio empleando el método apply(). Esto da lugar a una serie que indica la cantidad de palabras presentes en cada comentario.</p>
<p>Se procede a calcular el promedio de palabras presentes en los comentarios de cada grupo utilizando el método mean().</p>

 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 5.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál es el número promedio de oraciones en los comentarios de cada grupo?</span>

In [15]:

# Función para contar el número de oraciones en un comentario
def oracionesPorGrupo(comentario):
    doc = nlp(comentario)
    return len(list(doc.sents))

print("Promedio de oraciones en comentarios de odio:", 
      comentariosOdio['CONTENIDO A ANALIZAR TOKENIZED'].apply(oracionesPorGrupo).mean())
print("Promedio de oraciones en comentarios de no odio:", 
      comentariosNoOdio['CONTENIDO A ANALIZAR TOKENIZED'].apply(oracionesPorGrupo).mean())

Promedio de oraciones en comentarios de odio: 1.0671004473363155
Promedio de oraciones en comentarios de no odio: 1.1080751858306364


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
<p>Se realiza el cálculo del promedio de oraciones en los comentarios de dos grupos diferentes, aquellos que contienen expresiones de odio y aquellos que no.</p>
<p>Se ha establecido una función llamada oracionesPorGrupo(comentario)” que recibe un comentario como entrada, lo analiza utilizando el modelo PLN,especificado al principio, y cuenta la cantidad de oraciones en dicho comentario mediante la propiedad “sents” del objeto Doc en spaCy. Esta función devuelve el número total de oraciones presentes en el comentario.</p>
<p>Posteriormente, se aplica la función “oracionesPorGrupo” a la columna 'CONTENIDO A ANALIZAR TOKENIZED' en ambos DataFrames (comentariosOdio y comentariosNoOdio) utilizando el método “apply()". Esto genera una serie que refleja la cantidad de oraciones presentes en cada comentario.</p>
<p>Luego, se calcula el promedio de oraciones en los comentarios de cada grupo mediante el uso del método “mean()".</p>



<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 6.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál es el porcentaje de comentarios que contienen entidades NER en cada grupo?</span>

In [16]:

# Función para verificar si un comentario tiene entidades NER
def comentariosConNER(comentario):
    doc = nlp(comentario)
    return any(ent.label_ for ent in doc.ents)


print("Porcentaje de comentarios con entidades NER en comentarios de odio:", 
      ((comentariosOdio['CONTENIDO A ANALIZAR TOKENIZED'].apply(comentariosConNER).sum() / len(comentariosOdio)) * 100))
print("Porcentaje de comentarios con entidades NER en comentarios de no odio:", 
      ((comentariosNoOdio['CONTENIDO A ANALIZAR TOKENIZED'].apply(comentariosConNER).sum() / len(comentariosNoOdio)) * 100))

Porcentaje de comentarios con entidades NER en comentarios de odio: 40.30093533956893
Porcentaje de comentarios con entidades NER en comentarios de no odio: 56.18842693032685


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>

<p>Se crea una función llamada comentariosConNER(comentario)” que recibe un comentario como entrada, lo procesa mediante un modelo pln especificado al principio, y verifica la presencia de entidades NER en dicho comentario utilizando la propiedad “ents” del objeto Doc en spaCy. Esta función devuelve True si encuentra entidades NER en el comentario, y False si no las encuentra.</p>
<p>Posteriormente, se aplica la función “comentariosConNER” a la columna 'CONTENIDO A ANALIZAR TOKENIZED' tanto en los DataFrames correspondientes a los comentarios con odio como a los comentarios sin odio. Esto resulta en una serie que contiene valores booleanos indicando si cada comentario contiene entidades NER.</p>
<p>Finalmente, se calcula el porcentaje de comentarios que contienen entidades NER en cada grupo dividiendo el número total de comentarios que contienen entidades NER entre el número total de comentarios en ese grupo. El resultado se multiplica por 100 para obtener el porcentaje correspondiente.</p>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 7.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál es el porcentaje de comentarios que contienen entidades NER de tipo PERSON en cada grupo?</span>

In [17]:

# Función para verificar si un comentario contiene entidades NER de tipo PERSON
def cotienePersona(comentario):
    doc = nlp(comentario)
    return any(ent.label_ == "PER" for ent in doc.ents)

print("Porcentaje de comentarios con entidades NER de tipo PERSON en comentarios de odio:", 
      ((comentariosOdio['CONTENIDO A ANALIZAR TOKENIZED'].apply(cotienePersona).sum() / len(comentariosOdio)) * 100))
print("Porcentaje de comentarios con entidades NER de tipo PERSON en comentarios de no odio:", 
      ((comentariosNoOdio['CONTENIDO A ANALIZAR TOKENIZED'].apply(cotienePersona).sum() / len(comentariosNoOdio)) * 100))

Porcentaje de comentarios con entidades NER de tipo PERSON en comentarios de odio: 27.572183814558766
Porcentaje de comentarios con entidades NER de tipo PERSON en comentarios de no odio: 36.62108333036952


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>

<p>Se crea una función llamada “contienePersona(comentario)” que recibe como entrada un comentario, lo procesa mediante el modelo de pln, especificado al principio, y verifica si hay alguna entidad de tipo PERSON presente en el comentario utilizando la propiedad “ents” del objeto Doc de spaCy. Esta función devuelve Verdadero si encuentra entidades de tipo PERSON en el comentario, y Falso en caso contrario.</p>
<p>La función “contienePersona” se aplica a la columna 'CONTENIDO A ANALIZAR TOKENIZED' en ambos DataFrames. Esto produce una serie que contiene valores booleanos que señalan si cada comentario incluye entidades de tipo PERSON.</p>
<p>Se calcula el porcentaje de comentarios que contienen entidades de tipo PERSON en cada grupo dividiendo la cantidad de comentarios con entidades de tipo PERSON entre la cantidad total de comentarios en ese grupo, y multiplicando el resultado por 100 para obtener el porcentaje.</p>

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 8.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál es el porcentaje de palabras en cada combinación posible de género y número (p.ej. masculino singular) en cada grupo?</span>

In [18]:
from collections import Counter

# Función para contar el número de palabras en cada combinación posible de género y número
def contadorGeneroNumero(dfComentarios):

    combinaciones = Counter()

    for comentario in dfComentarios:
        doc = nlp(comentario)

        for token in doc:
            if token.pos_ == "NOUN":
                generoNumero = (token.morph.get("Gender", "N")[0], token.morph.get("Number", "S")[0])
                combinaciones[tuple(generoNumero)] += 1

    totalPalabras = sum(combinaciones.values())
    pctTotalPalabras = {generoNumero: (contador / totalPalabras) * 100 for generoNumero, contador in combinaciones.items()}
    
    return pctTotalPalabras

pctCombinacionesOdio = contadorGeneroNumero(comentariosOdio['CONTENIDO A ANALIZAR TOKENIZED'])
pctCombinacionesNoOdio = contadorGeneroNumero(comentariosNoOdio['CONTENIDO A ANALIZAR TOKENIZED'])

print("Porcentaje de palabras en cada combinación posible en comentarios de odio:")
for clave, valor in pctCombinacionesOdio.items():
    print(clave, ';', valor)

print("\nPorcentaje de palabras en cada combinación posible en comentarios de no odio:")
for clave, valor in pctCombinacionesNoOdio.items():
    print(clave, ';', valor)

Porcentaje de palabras en cada combinación posible en comentarios de odio:
('N', 'S') ; 12.788751255440243
('Masc', 'Sing') ; 41.345831938399726
('Fem', 'Sing') ; 39.772346836290595
('N', 'Sing') ; 3.7495815199196514
('Masc', 'Plur') ; 0.920656176765986
('Fem', 'Plur') ; 0.5691329092735186
('Fem', 'S') ; 0.36826247070639434
('Masc', 'S') ; 0.4017408771342484
('N', 'Plur') ; 0.08369601606963509

Porcentaje de palabras en cada combinación posible en comentarios de no odio:
('Masc', 'Sing') ; 42.092997105402524
('Fem', 'Sing') ; 37.97721864730164
('N', 'S') ; 15.546083225118432
('Fem', 'S') ; 0.42938342897354237
('N', 'Sing') ; 2.270109835102254
('Fem', 'Plur') ; 0.45182744136445085
('Masc', 'Plur') ; 0.7270046357087412
('Masc', 'S') ; 0.4685584324194917
('N', 'Plur') ; 0.036817248608924644


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>

<p>Se crea una función llamada contadorGeneroNumero(dfComentarios) que recibe como entrada una serie de comentarios y cuenta la cantidad de palabras en cada combinación posible de género y número. Se emplea un objeto Counter para llevar a cabo este conteo.</p>
<p>Para cada comentario, se procesa con PLN y se recorre los tokens. Si un token es un sustantivo (NOUN), se extrae su género y número a través de sus propiedades morfológicas (morph.get("Gender") y morph.get("Number")) para luego añadirlo al contador combinaciones.</p>
<p>Se calcula el total de palabras en todas las combinaciones, así como el porcentaje correspondiente a cada combinación posible.</p>


 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 9.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio), indica cuántas entidades de cada tipo posible se reconocen en cada uno de los grupos</span>

In [23]:
# Función para contar el número de entidades de cada tipo posible en un grupo de comentarios
def entidadesPorGrupo(dfComentarios):
    contadorEntidades = Counter()
    for comentario in dfComentarios:
        doc = nlp(comentario)
        for ent in doc.ents:
            contadorEntidades[ent.label_] += 1
    return contadorEntidades


print("Entidades por tipo en comentarios de odio:")
print(entidadesPorGrupo(comentariosOdio['CONTENIDO A ANALIZAR TOKENIZED']))
print("\nEntidades por tipo en comentarios de no odio:")
print(entidadesPorGrupo(comentariosNoOdio['CONTENIDO A ANALIZAR TOKENIZED']))


Entidades por tipo en comentarios de odio:
Counter({'PER': 862, 'MISC': 222, 'LOC': 178, 'ORG': 102})

Entidades por tipo en comentarios de no odio:
Counter({'PER': 158435, 'LOC': 72165, 'MISC': 42466, 'ORG': 31312})


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>

<p>Se crea una función llamada entidadesPorGrupo(dfComentarios) que recibe una serie de comentarios como entrada y cuenta cuántas entidades hay en cada categoría posible.</p>
<p>Para cada comentario, se aplica el análisis con nlp y se recorre las entidades reconocidas en dicho comentario (doc.ents). Por cada entidad, se incrementa el contador correspondiente al tipo de entidad (ent.label_) en el objeto contadorEntidades.</p>
<p>Se invoca la función entidadesPorGrupo tanto para los comentarios relacionados con sentimientos negativos, como para los comentarios sin relación con sentimientos negativos</p>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 10.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio), extrae y muestra los 100 lemas más repetidos en los comentarios de cada grupo</span>

In [24]:

# Función para obtener los lemas de los tokens en un comentario
def obtenerLemas(comentario):
    doc = nlp(comentario)
    
    return [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

# Función para obtener los 100 lemas más comunes en un grupo de comentarios
def lemasComunes(dfComentarios):
    lemas = []

    for comentario in dfComentarios:
        lemas.extend(obtenerLemas(comentario))
    contadorLemas = Counter(lemas)

    return contadorLemas.most_common(100)


lemas_odio = lemasComunes(comentariosOdio['CONTENIDO A ANALIZAR TOKENIZED'])
lemas_no_odio = lemasComunes(comentariosNoOdio['CONTENIDO A ANALIZAR TOKENIZED'])

print("100 lemas más comunes en comentarios de odio:")
for lema, cantidad in lemas_odio:
    print(lema, "-", cantidad)

print("\n100 lemas más comunes en comentarios de no odio:")
for lema, cantidad in lemas_no_odio:
    print(lema, "-", cantidad)

100 lemas más comunes en comentarios de odio:
puta - 137
informativo - 124
españa - 121
hijo - 121
asco - 120
mierda - 116
gobierno - 112
español - 110
mierdar - 93
basura - 86
terrorismo - 85
q - 82
gente - 80
país - 77
tonto - 70
pagar - 64
político - 62
dejar - 62
terrorista - 61
vergüenza - 59
gentuza - 56
panfleto - 54
madrid - 53
puto - 53
miserable - 52
madre - 51
culo - 51
putar - 51
salir - 50
pasar - 49
año - 47
mundo - 47
mentiroso - 47
comunista - 46
mujer - 42
asqueroso - 42
i - 41
cárcel - 40
catalán - 40
ja - 40
importar - 39
noticia - 39
vox - 36
vida - 36
pensar - 35
dinero - 33
persona - 33
dais - 33
querer - 33
gilipol - 33
sánchez - 32
fascista - 32
racista - 32
derecho - 31
seguir - 31
facha - 31
calle - 31
esperar - 30
llamar - 29
cobarde - 29
tomar - 29
tiempo - 29
creer - 29
asesino - 29
votar - 29
llegar - 28
partido - 28
cosa - 28
deber - 28
per - 28
maldito - 28
catalanisme - 27
venir - 27
hombre - 27
trump - 27
delincuente - 26
hablar - 26
quedar - 26
periód

<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
<p>Se crea una función llamada obtenerLemas(comentario) que recibe un comentario como entrada y retorna una lista de los lemas de las palabras en el comentario que son alfabéticas y no son stopwords.</p>
<p>Se establece otra función denominada lemasComunes(dfComentarios) que recibe una serie de comentarios como entrada. Dentro de esta función, se itera sobre cada comentario en el grupo correspondiente. Para cada comentario, se obtienen los lemas utilizando la función obtenerLemas y se añaden a una lista llamada lemas.</p> <p>Posteriormente, se emplea un objeto Counter para contar la frecuencia de cada lema en la lista generada. Finalmente, se presentan los 100 lemas más comunes utilizando el método most_common(100) del objeto Counter.
<p>Se invoca la función lemasComunes para analizar tanto los comentarios asociados al odio, como a los de sin odio.</p>

 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 11.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Es posible utilizar alguna de las características extraídas en las preguntas anteriores para determinar si un mensaje contiene odio? Justifica tu respuesta con el análisis estadístico que consideres necesario.</span>

<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
<p>Sí, se puedeun utilizar algunas de las características extraídas en las cuestiones anteriores para determinar si un comentario contiene odio o no. A continuación se detallan 3 casos de interés: </p>

<ul>
  <li><p>Porcentaje de comentarios que incluyen entidades NER: También notamos que la proporción de comentarios que contienen entidades NER en los comentarios negativos es menor en comparación con los comentarios que no están relacionados con el odio. Esto podría sugerir que los mensajes de odio tienden a tener menos información detallada o menos menciones a entidades específicas.</p></li>

  <li><p>Porcentaje de comentarios que contienen entidades NER del tipo PERSONA: De manera similar al punto anterior, la proporción de comentarios que contienen entidades NER del tipo PERSONA es menor en los comentarios de odio en comparación con los comentarios no relacionados con el odio. Esto podría indicar una menor frecuencia de menciones específicas a personas en los mensajes de odio.</p></li>

  <li><p>Los lemas más frecuentes en los comentarios negativos pueden ser distintos a aquellos presentes comúnmente en los comentarios sin relación al odio. Por ejemplo, ciertas expresiones ofensivas o términos específicos podrían ser más recurrentes en los mensajes cargados de odio.</p></li>

</ul>
 